<a href="https://colab.research.google.com/github/salvatore-dacunto/beam_demo/blob/master/Beam_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip3 install apache_beam